# Combined hazard damage relationship

This notebook explores the relationship between reported damage and a combination of wind and rainfall as hazard parameters. 


In [ ]:
%matplotlib notebook

import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import seaborn as sns


In [ ]:
data_path = "N:/georisk/HaRIA_B_Wind/projects/impact_forecasting/data/exposure/NSW/April_2015_Impact_Assessment"
data_path = "C:/WorkSpace/data/derived/exposure/NSW"
filename = "Property_damage_cleaned.shp" 
filepath = pjoin(data_path, filename)

In [ ]:
gdf = gpd.read_file(filepath)

In [ ]:
gdf.columns

In [ ]:
gdf[['EICU_Degda', 'maxwind','rainrate']]

In [ ]:
gdf['EICU_Degda'].unique()

In [ ]:
DMG_ORDER=['No Damage - 0%','Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%','Destroyed - 76-100%']
sns.lmplot(x='maxwind', y='rainrate',hue='EICU_Degda', data=gdf, fit_reg=False, aspect=1.5,hue_order=DMG_ORDER)

In [ ]:
f, axes = plt.subplots(2,3,sharex=True, sharey=True)

for ax, dmgstate in zip(axes.flat, DMG_ORDER[0:]):
    df = gdf[gdf['EICU_Degda']==dmgstate][['maxwind', 'rainrate']]
    sns.kdeplot(df['maxwind'], df['rainrate']**2, ax=ax)
    n = len(df)
    ax.set_title("{0} (n={1})".format(dmgstate, n), fontsize='small')
    ax.set_xlim((0, 20))
    ax.set_ylim((0, 15000))
f.tight_layout()

So it looks like adding maximum rainfall rate (kg/m2/s) as a hazard parameter will not improve the discrimination of damage states. The peak density for the destroyed buildings (n=8) overlaps with the no damage density (n=1465), and is almost colocated with the peak for minor impact.

Another option would be to explore accumulated rainfall amounts.

In [ ]:
gdf['roofdam'].unique()

In [ ]:
ROOF_DMG_ORDER = ['Stable','Minor','Moderate','Severe']
sns.lmplot(x='maxwind', y='rainrate',hue='roofdam', data=gdf, fit_reg=False, aspect=1,hue_order=ROOF_DMG_ORDER)

In [ ]:
f, axes = plt.subplots(2,2,sharex=True, sharey=True)

for ax, dmgstate in zip(axes.flat, ROOF_DMG_ORDER):
    df = gdf[gdf['roofdam']==dmgstate][['maxwind', 'rainrate']]
    n = len(df)
    sns.jointplot(df['maxwind'], df['rainrate'],kind='kde')# ax=ax)
    ax.set_title("{0} (n={1})".format(dmgstate, n))
    ax.set_xlim((0, 20))
    ax.set_ylim((0, 150))
f.tight_layout()

In [ ]:
gdf.groupby([ 'waterinund','roofdam','EICU_Degda']).agg('mean')['rainrate']

In [ ]:
gdf.groupby([ 'waterinund','roofdam','EICU_Degda']).agg('mean')['maxwind']

In [ ]:
gdf['EICU_Degda'].isin(['Severe Impact - 51-75%','Major Impact - 26-50%','Destroyed - 76-100%'])